![Python Logo](./img/python-logo-master-v3-TM-flattened_small.png)

# Financial Programming - IÉSEG, Lille - MBD 2019-2020
## Group Project - Financial Data

In [2]:
first_name = 'Clement'
last_name = 'Bannem'

### Data Exploration

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from pandasql import sqldf 
pysqldf = lambda q: sqldf(q, globals())

**Input data**

In [5]:
# Import banking data
# Note: This data was extracted on 1999

# Each record describes characteristics of a client
client = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/client.asc', sep=';')

# Each record describes static characteristics of an account
account = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/account.asc', sep=';')

# Each record describes a credit card issued to an account
card = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/card.asc', sep=';')

# Each record describes demographic characteristics of a district
district = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/district.asc', sep=';')

# Each record relates together a client with an account
# i.e. this relation describes the rights of clients to operate accounts
disp = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/disp.asc', sep=';')

# Each record describes characteristics of a payment order (debits only)
order = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/order.asc', sep=';')

# Each record describes one transaction on an account
trans = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/trans.asc', sep=';', low_memory=False)

# Each record describes a loan granted for a given account
loan = pd.read_csv('C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/7. Financial Programming/Group Project/Group Project_Customer Loan/data_berka/loan.asc', sep=';')

## Adding information to each record, replacing obscure values.

#### Client record

In [6]:
#Adding birthyear and Age to the client data base
client['birth_year'] = client['birth_number'].transform(lambda bn: int('19' + str(bn)[:2]))
client['age'] = 1999 - client['birth_year']

In [7]:
#Extracting gender from client[birth_number] and reformating it
# Function to extract birth month and gender
def to_month_gender(birth_number):
    
    # Extract the birth month and birth day from the birth_number column
    mon = str(birth_number)
    birth_month = int(mon[2:4])
    day = str(birth_number)
    birth_day = int(day[4:])
    
    # Extract the gender from the birth month
    if birth_month > 50:
        gender = "F"
        birth_month = birth_month - 50
    else:
        gender = 'M'
        
    return pd.Series({'birth_month':birth_month, 'birth_day':birth_day, 'gender':gender})

# Apply the function to client table
client[['birth_month', 'birth_day', 'gender']] = client['birth_number'].apply(to_month_gender)

In [8]:
client.head()

,client_id,birth_number,district_id,birth_year,age,birth_month,birth_day,gender
0,1,706213,18,1970,29,12,13,F
1,2,450204,1,1945,54,2,4,M
2,3,406009,1,1940,59,10,9,F
3,4,561201,5,1956,43,12,1,M
4,5,605703,5,1960,39,7,3,F


#### Account record

In [9]:
account.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [10]:
def replfreq (freqcode):
    if "MESICNE" in freqcode:
        freq = "MONTHLY"
    elif "TYDNE" in freqcode:
        freq = "WEEKLY"
    elif "OBRATU" in freqcode:
        freq = "AFTER TRANSACTION"
    
    return freq

def repldate (datecode): #replace the YYMMDD column with 3 columns for Year (YYYY) Month and Day
    Year = "19" + str(datecode)[:2]
    Month = str(datecode)[2:4]
    Day = str(datecode)[4:6]
    
    return pd.Series({'Year':Year,'Month':Month,'Day':Day})

In [11]:
account['freq'] = account["frequency"].apply(replfreq)

In [12]:
account[['Year','Month','Day']] = account["date"].apply(repldate)

In [13]:
account.head()

,account_id,district_id,frequency,date,freq,Year,Month,Day
0,576,55,POPLATEK MESICNE,930101,MONTHLY,1993,01,01
1,3818,74,POPLATEK MESICNE,930101,MONTHLY,1993,01,01
2,704,55,POPLATEK MESICNE,930101,MONTHLY,1993,01,01
3,2378,16,POPLATEK MESICNE,930101,MONTHLY,1993,01,01
4,2632,24,POPLATEK MESICNE,930102,MONTHLY,1993,01,02


#### Order record

In [14]:
order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


In [15]:
def krepl (k): #replace the k_symbol by a more comprehensive string (used in multiple records)
    k = str(k)
    if 'POJISTNE' in k:
        payment = "INSURRANCE"
    elif 'SIPO' in k:
        payment = "HOUSEHOLD"
    elif 'UVER' in k:
        payment = "LOAN"
    elif 'LEASING' in k:
        payment = "LEASING"
    elif 'SLUZBY' in k:
        payment = "STATEMENT"
    elif "UROK" in k:
        payment = "INTEREST CREDITED"
    elif 'SANKC. UROK' in k:
        payment = "SANCTION INTEREST (NEGATIVE BALANCE)"
    elif 'DUCHOD' in k :
        payment = "OLD AGE PENSION"
    else:
        payment = "UNKNOWN"
    
    return payment

In [16]:
order["Payment characterization"] = order["k_symbol"].apply(krepl)

In [17]:
order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol,Payment characterization
0,29401,1,YZ,87144583,2452.0,SIPO,HOUSEHOLD
1,29402,2,ST,89597016,3372.7,UVER,LOAN
2,29403,2,QR,13943797,7266.0,SIPO,HOUSEHOLD
3,29404,3,WX,83084338,1135.0,SIPO,HOUSEHOLD
4,29405,3,CD,24485939,327.0,,UNKNOWN


#### Transaction record

In [18]:
print(trans.shape)
trans.head()

(1056320, 10)


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


##### This record is more than a million row long. Any operation on this record will take some time, please process it carrefully.

In [19]:
#trans[['Year','Month','Day']] = trans["date"].apply(repldate)

In [20]:
def typerepl(Type):
    if "PRIJEM" in Type:
        transaction = "+"
    else:
        transaction = "-"
    return transaction

def operepl(ope):
    ope = str(ope)
    if "VYBER KARTOU" in ope:
        mode = "Withdrawal in C.C."
    elif "VKLAD" in ope:
        mode = "Credit in cash"
    elif "PREVOD Z UCTU" in ope:
        mode = "Withdrawal from another bank"
    elif "VYBER" in ope:
        mode = "Withdrawal in cash"
    elif "PREVOD NA UCET" in ope:
        mode = "Credit to another bank"
    else:
        mode = "UNKNOWN"
    return mode

In [21]:
trans["Payment Characterization"] = trans["k_symbol"].apply(krepl)
trans["Transaction Type"] = trans["type"].apply(typerepl)
trans["Transaction Mode"] = trans["operation"].apply(operepl)

In [22]:
trans.head(20)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,Payment Characterization,Transaction Type,Transaction Mode
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
5,771035,2632,930102,PRIJEM,VKLAD,1100.0,1100.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
6,452728,1539,930103,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
7,725751,2484,930103,PRIJEM,VKLAD,1100.0,1100.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
8,497211,1695,930103,PRIJEM,VKLAD,200.0,200.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
9,232960,793,930103,PRIJEM,VKLAD,800.0,800.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash


In [23]:
trans.operation.isnull().sum()

183114

#### Loan record

In [24]:
loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


In [25]:
loan[['Year','Month','Day']] = loan['date'].apply(repldate)

In [26]:
loan.head()

,loan_id,account_id,date,amount,duration,payments,status,Year,Month,Day
0,5314,1787,930705,96396,12,8033.0,B,1993,07,05
1,5316,1801,930711,165960,36,4610.0,A,1993,07,11
2,6863,9188,930728,127080,60,2118.0,A,1993,07,28
3,5325,1843,930803,105804,36,2939.0,A,1993,08,03
4,7240,11013,930906,274740,60,4579.0,A,1993,09,06


#### Card record

In [27]:
card.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,931107 00:00:00
1,104,588,classic,940119 00:00:00
2,747,4915,classic,940205 00:00:00
3,70,439,classic,940208 00:00:00
4,577,3687,classic,940215 00:00:00


In [28]:
card[["Year","Month","Day"]] = card["issued"].apply(repldate)

In [29]:
card.head()

,card_id,disp_id,type,issued,Year,Month,Day
0,1005,9285,classic,931107 00:00:00,1993,11,07
1,104,588,classic,940119 00:00:00,1994,01,19
2,747,4915,classic,940205 00:00:00,1994,02,05
3,70,439,classic,940208 00:00:00,1994,02,08
4,577,3687,classic,940215 00:00:00,1994,02,15


#### District record

In [30]:
district.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [31]:
district.columns = ['district_code', 'district_name', 'region', 'Number_inhabitants', 'A5', 'A6', 'A7', 'A8', 'A9', '%_urbans', 'Avrg_Salary',
       'A12', 'A13', 'A14', 'A15', 'A16']

### Interactive Tool

Variable to create for credit scoring :


In [32]:
def loansituation(n):
    global paid,nbrpayment,lefttopay #
    
    #Subsetting the main table to have only transaction linked to loans
    IDinfoLoan = IDinfo[IDinfo["Payment Characterization"]=="LOAN"]
    
    #initiating in this order : number of transaction linked to loans, amount paid on the specific loan, nbr of payment,
    #amount left to pay for this specific loan
    testlen = len(IDinfoLoan)
    paid =0
    nbrpayment=0
    lefttopay = loantest.amount.iloc[n]
    
    for i in range(testlen):
        if loantest.payments.iloc[0]==round(IDinfoLoan["amount"].iloc[i]): #if this is true, it is a payment corresponding
            paid += round(IDinfoLoan["amount"].iloc[i])                    #to the specific loan we're looking at
            nbrpayment += 1
            lefttopay -= round(IDinfoLoan["amount"].iloc[i])
        else:
            None
    


def genID():
    global clientest, disptest, IDinfo, ID, districtest, transtest, loantest, cardtest #mainly for testing and debugging purpose
    
    #Prompting the user to enter an ID
    ID = int(input("Which client ID would you like to investigate ? \n Please type the ID here : "))
    clientest = client[client["client_id"]==ID]    
    disptest = disp[disp["client_id"]==ID]
    
    #Merging client and disp to get the account_id with client info
    IDinfo = pd.merge(clientest[["client_id","district_id","age","birth_month","gender"]],
                      disptest,
                      how="inner",
                      left_on="client_id",
                      right_on="client_id"
                     )
    
    #ordering the columns in a more comprehensive way
    IDinfo = IDinfo[["client_id","account_id","type","disp_id","age","birth_month","gender","district_id"]]
    
    #taking the district table based on client ID required
    districtest = district[district["district_code"]==IDinfo.district_id[0]] 
    
    #Adding district name and average Salary of this district to our main table
    IDinfo = pd.merge(IDinfo,    
                      districtest [["district_code","district_name","Avrg_Salary"]],
                      how="inner",
                      left_on="district_id",
                      right_on="district_code"
                     ) 
     
    #Taking transaction record of the client
    transtest = trans[trans["account_id"]==IDinfo["account_id"].iloc[0]]
    
    #Taking information about the potential credit card of the customer
    cardtest = card[card["disp_id"]==IDinfo["disp_id"].iloc[0]]
    
    #Adding those informations to our main table
    IDinfo = pd.merge(IDinfo,
                      transtest [["account_id","amount","balance","Year","Month","Day","Payment Characterization","Transaction Mode"]],
                      how="inner",
                      left_on="account_id",
                      right_on="account_id"
                     ) 
    
    #Taking information about the account of the client
    accountest = account[account["account_id"]==IDinfo["account_id"].iloc[0]]
    
    #last will be used to take the last row of the main table, to identify the last transaction registered
    last = len(IDinfo)-1
    
    #Loading the loan informations of the client
    loantest = loan[loan["account_id"]==IDinfo["account_id"].iloc[0]]
    
    #lenght of this table gives the number of loans taken
    nbofloan = len(loantest)
    
    
    ###PRINTING###
    
    #printing general information about the client
    print("-"*20)
    print("\nClient is a " + IDinfo.gender[0] + ", " + str(IDinfo.age[0]) + " years old and lives in " + 
          str(IDinfo.district_name[0]) + ".")
    print("\nHas been a client since " + accountest.Day.iloc[0]+"/"+
        str(accountest.Month.iloc[0])+"/"+str(accountest.Year.iloc[0]))
    if len(cardtest) == 0:
        print("\nThis client has no credit card in our bank.")
    else:
        for n in range(len(cardtest)):
            print("and was issued a " + str(cardtest.type.iloc[n]) + " credit card in " +
                  str(accountest.Day.iloc[n]) + "/" + str(accountest.Month.iloc[n]) + "/" + str(accountest.Year.iloc[n]) + ".")
    
    print("\nLast transaction registered occured the  " +
          str(IDinfo.Day[last])+"/"+str(IDinfo.Month[last])+"/"+str(IDinfo.Year[last]) + 
          ". \nBank account balance : " + str(IDinfo.balance[last])+".")
    
    #printing information about the last non-automatic transaction (no interest credit) to better identify
    #if the client is a sleeping account or not
    while IDinfo["Payment Characterization"][last]=="INTEREST CREDITED":
        last -=1
    
    print("\nLast transaction that was not automatic was " +str(IDinfo["Payment Characterization"][last])+ 
          "(" +str(IDinfo["Transaction Mode"][last]) + ")"" and occured in " +
          str(IDinfo.Day[last])+"/"+str(IDinfo.Month[last])+"/"+str(IDinfo.Year[last] ))
    
    
    #Printing information about the loans (their number, id, status, date of last payment if it has been fully payed,
    #amount left to pay, montly payment, time left before fully paid if everythin goes right.)
    if nbofloan == 0:
        print("\nThis client never took any loan from us.")
    
    else:
        print("\nThis client was issued " + str(nbofloan) + " loan(s). \nLoan status is/are :\n")
        
        for n in range(nbofloan):
            print("*Loan " +str(loantest["loan_id"].iloc[n]) + " issued the " + 
                  str(loantest.Day.iloc[n-1])+"/"+str(loantest.Month.iloc[n-1])+"/"+str(loantest.Year.iloc[n-1]) +
                  " for an amount of " + str(loantest.amount.iloc[n-1]) +
                  " and its status is " + str(loantest.status.iloc[n-1])+".")
            
            if loantest.status.iloc[n-1] =='A':
                loansituation(n)
                print("\n\tThe loan has been fully paid.")
            
            elif loantest.status.iloc[n] =='D':
                print("\n\tClient is late in the payment of the loan, the client is in debt !")
                loansituation(n)
                print("\n\tAmount already paid : " + str(paid) + 
                      "\n\tAmount left to pay : " + str(lefttopay) +
                      "\n\tNumber of payment made : " + str(nbrpayment))
                print("\n\t")
            
            elif loantest.status.iloc[n] =='B':
                print("\n\tThe contract is finished but the loan is not fully paid yet !")
                loansituation(n)
                print("\n\tAmount already paid : " + str(paid) + 
                      "\n\tAmount left to pay : " + str(lefttopay) +
                      "\n\tNumber of payment made : " + str(nbrpayment)) 
                
                IdinfoLoan = IDinfo[IDinfo["Payment Characterization"]=="LOAN"]
                loantest.payments[n]==IDinfoLoan[IDinfoLoan["amount"]]
                
            elif loantest.status.iloc[n] =='C':
                print("\n\tThe contract is still going and the loan is not fully paid yet. Situation is normal.")
                loansituation(n)
                print("\n\tAmount already paid : " + str(paid) + 
                      "\n\tAmount left to pay : " + str(lefttopay) +
                      "\n\tNumber of payment made : " + str(nbrpayment)) 
                
            else:
                print("\n\tThe status of this loan seems to been unknown or incorrectly specified in our files." +
                     "\n\tAsk IT for support on this problem.")
                
    print("\n" + "-"*20)
    #return IDinfo.tail()

In [33]:
# genID()

In [34]:
#To create a field to merge on 
card['account_id'] = card['disp_id']
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 8 columns):
card_id       892 non-null int64
disp_id       892 non-null int64
type          892 non-null object
issued        892 non-null object
Year          892 non-null object
Month         892 non-null object
Day           892 non-null object
account_id    892 non-null int64
dtypes: int64(3), object(5)
memory usage: 55.8+ KB


In [35]:
#To merge card and trans: created to see which card carried out which transaction.
card_transaction = pd.merge(card, trans, how = 'left', on = 'account_id')
card_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141244 entries, 0 to 141243
Data columns (total 20 columns):
card_id                     141244 non-null int64
disp_id                     141244 non-null int64
type_x                      141244 non-null object
issued                      141244 non-null object
Year                        141244 non-null object
Month                       141244 non-null object
Day                         141244 non-null object
account_id                  141244 non-null int64
trans_id                    140967 non-null float64
date                        140967 non-null float64
type_y                      140967 non-null object
operation                   116526 non-null object
amount                      140967 non-null float64
balance                     140967 non-null float64
k_symbol                    78140 non-null object
bank                        37673 non-null object
account                     39819 non-null float64
Payment Characterizatio

In [36]:
#To change name
client["district_code"] = client["district_id"]
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 9 columns):
client_id        5369 non-null int64
birth_number     5369 non-null int64
district_id      5369 non-null int64
birth_year       5369 non-null int64
age              5369 non-null int64
birth_month      5369 non-null int64
birth_day        5369 non-null int64
gender           5369 non-null object
district_code    5369 non-null int64
dtypes: int64(8), object(1)
memory usage: 377.6+ KB


In [37]:
#start of basetable1
client_district = pd.merge(client, district, on = 'district_code', how = 'left')
#To set index
client_district.set_index('district_id')
client_district.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 24 columns):
client_id             5369 non-null int64
birth_number          5369 non-null int64
district_id           5369 non-null int64
birth_year            5369 non-null int64
age                   5369 non-null int64
birth_month           5369 non-null int64
birth_day             5369 non-null int64
gender                5369 non-null object
district_code         5369 non-null int64
district_name         5369 non-null object
region                5369 non-null object
Number_inhabitants    5369 non-null int64
A5                    5369 non-null int64
A6                    5369 non-null int64
A7                    5369 non-null int64
A8                    5369 non-null int64
A9                    5369 non-null int64
%_urbans              5369 non-null float64
Avrg_Salary           5369 non-null int64
A12                   5369 non-null object
A13                   5369 non-null float64
A14

In [38]:
#drop added filed for merge
client_district = client_district.drop(['district_code'], axis = 1)
#drop unwanted columns
client_district = client_district.drop(columns = ['A5','A6','A7','A8','A9','A12','A13','A14','A15','A16'])

client_district.head()

,client_id,birth_number,district_id,birth_year,age,birth_month,birth_day,gender,district_name,region,Number_inhabitants,%_urbans,Avrg_Salary
0,1,706213,18,1970,29,12,13,F,Pisek,south Bohemia,70699,65.3,8968
1,2,450204,1,1945,54,2,4,M,Hl.m. Praha,Prague,1204953,100.0,12541
2,3,406009,1,1940,59,10,9,F,Hl.m. Praha,Prague,1204953,100.0,12541
3,4,561201,5,1956,43,12,1,M,Kolin,central Bohemia,95616,51.4,9307
4,5,605703,5,1960,39,7,3,F,Kolin,central Bohemia,95616,51.4,9307


In [39]:
# Use so that each observation is a client 
client_district.set_index('client_id')

,birth_number,district_id,birth_year,age,birth_month,birth_day,gender,district_name,region,Number_inhabitants,%_urbans,Avrg_Salary
client_id,,,,,,,,,,,,
1,706213,18,1970,29,12,13,F,Pisek,south Bohemia,70699,65.3,8968
2,450204,1,1945,54,2,4,M,Hl.m. Praha,Prague,1204953,100.0,12541
3,406009,1,1940,59,10,9,F,Hl.m. Praha,Prague,1204953,100.0,12541
4,561201,5,1956,43,12,1,M,Kolin,central Bohemia,95616,51.4,9307
5,605703,5,1960,39,7,3,F,Kolin,central Bohemia,95616,51.4,9307
6,190922,12,1919,80,9,22,M,Pribram,central Bohemia,107870,58.0,8754
7,290125,15,1929,70,1,25,M,Cesky Krumlov,south Bohemia,58796,51.9,9045
8,385221,51,1938,61,2,21,F,Trutnov,east Bohemia,121947,70.5,8541
9,351016,60,1935,64,10,16,M,Prostejov,south Moravia,110643,51.9,8441


In [40]:
#change name
basetable1 = client_district  

In [41]:
#check for nulls
basetable1.isna().sum().sum()

0

In [42]:
#to drop
#basetable1 = basetable1.drop(columns = ['birth_number'])
#set index on new table
basetable1.set_index('client_id')
basetable1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 13 columns):
client_id             5369 non-null int64
birth_number          5369 non-null int64
district_id           5369 non-null int64
birth_year            5369 non-null int64
age                   5369 non-null int64
birth_month           5369 non-null int64
birth_day             5369 non-null int64
gender                5369 non-null object
district_name         5369 non-null object
region                5369 non-null object
Number_inhabitants    5369 non-null int64
%_urbans              5369 non-null float64
Avrg_Salary           5369 non-null int64
dtypes: float64(1), int64(9), object(3)
memory usage: 587.2+ KB


In [43]:
#transferred_out_amounts = order.groupby('account_id')["amount"].agg("sum")
#order["transferred_out_amounts"] = order["account_id"].map(transferred_out_amounts)
#order

In [44]:
def disponent (i):
        if i == "DISPONENT":
            return "Y"
        else:
            return "N"
        

In [45]:
Table1 = pd.merge(basetable1,disp, how = 'left', on = 'client_id')

Table2 = pd.merge(Table1,account, how = 'left', on = 'account_id')

Table3 = pd.merge(Table2,loan, how = 'left', on = 'account_id')

#Table4 = pd.merge(Table3,order, how ='left', on = 'account_id')

In [46]:
Table3["Disponent"] = Table3["type"].apply(disponent)
Table3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 33 columns):
client_id             5369 non-null int64
birth_number          5369 non-null int64
district_id_x         5369 non-null int64
birth_year            5369 non-null int64
age                   5369 non-null int64
birth_month           5369 non-null int64
birth_day             5369 non-null int64
gender                5369 non-null object
district_name         5369 non-null object
region                5369 non-null object
Number_inhabitants    5369 non-null int64
%_urbans              5369 non-null float64
Avrg_Salary           5369 non-null int64
disp_id               5369 non-null int64
account_id            5369 non-null int64
type                  5369 non-null object
district_id_y         5369 non-null int64
frequency             5369 non-null object
date_x                5369 non-null int64
freq                  5369 non-null object
Year_x                5369 non-null object
Mo

In [47]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 8 columns):
card_id       892 non-null int64
disp_id       892 non-null int64
type          892 non-null object
issued        892 non-null object
Year          892 non-null object
Month         892 non-null object
Day           892 non-null object
account_id    892 non-null int64
dtypes: int64(3), object(5)
memory usage: 55.8+ KB


In [48]:
trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,Payment Characterization,Transaction Type,Transaction Mode
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN,UNKNOWN,+,Credit in cash


In [49]:
Table3.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 33 columns):
client_id             5369 non-null int64
birth_number          5369 non-null int64
district_id_x         5369 non-null int64
birth_year            5369 non-null int64
age                   5369 non-null int64
birth_month           5369 non-null int64
birth_day             5369 non-null int64
gender                5369 non-null object
district_name         5369 non-null object
region                5369 non-null object
Number_inhabitants    5369 non-null int64
%_urbans              5369 non-null float64
Avrg_Salary           5369 non-null int64
disp_id               5369 non-null int64
account_id            5369 non-null int64
type                  5369 non-null object
district_id_y         5369 non-null int64
frequency             5369 non-null object
date_x                5369 non-null int64
freq                  5369 non-null object
Year_x                5369 non-null object
Mo

In [50]:
order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol,Payment characterization
0,29401,1,YZ,87144583,2452.0,SIPO,HOUSEHOLD
1,29402,2,ST,89597016,3372.7,UVER,LOAN
2,29403,2,QR,13943797,7266.0,SIPO,HOUSEHOLD
3,29404,3,WX,83084338,1135.0,SIPO,HOUSEHOLD
4,29405,3,CD,24485939,327.0,,UNKNOWN


In [51]:
order['bank_to'].nunique()

13

In [52]:
orders_copy = order

In [53]:
order_a = pysqldf("""
SELECT account_id,
sum(amount) as order_amount, 
count(amount) as number_of_orders
FROM orders_copy
GROUP BY account_id
""")

In [54]:
order_a.head()

,account_id,order_amount,number_of_orders
0,1,2452.0,1
1,2,10638.7,2
2,3,5001.0,3
3,4,3363.0,2
4,5,2668.0,1


In [55]:
trans_a = trans

In [56]:
trans_a =  pysqldf("""
SELECT account_id, operation,
sum(amount) as amount,
count(amount) as number_of_transaction,
Max(amount) as Max_transaction,
Min(amount) as Min_transaction,
avg(amount) as AVG_transaction,
            CASE 
                WHEN operation = 'VYBER KARTOU' THEN 'Credit Card Withdrawl'
                WHEN operation = 'VKLAD' THEN ' Credit in cash '
                WHEN operation = 'PREVOD Z UCTU' THEN 'Collection from another bank'
                WHEN operation = 'VYBER' THEN 'Withdrawl in cash'
                WHEN operation = 'PREVOD NA UCET' THEN 'Remittance to another banl'
                ELSE 'Other transaction'
            END AS TransactionMode,
            CASE 
                WHEN k_symbol = 'POJISTNE' THEN 'Insurance Payment'
                WHEN k_symbol = 'SLUZBY' THEN 'Payment for statement'
                WHEN k_symbol = 'UROK' THEN 'Intrest credited'
                WHEN k_symbol = 'SANKC. UROK' THEN 'Sanction interest if negative balance'
                WHEN k_symbol = 'SIPO' THEN 'Stands for household'
                WHEN k_symbol = 'DUCHOD' THEN 'Stands for old-age pension'
                WHEN k_symbol = '"UVER' THEN 'Stands for loan payment'
                ELSE 'Other'
            END AS TransactionCharacterization,
            SUM(CASE 
                    WHEN type = "PRIJEM" THEN 1 ELSE 0 END) as 'UsedForCredit',
            SUM(CASE 
                    WHEN type = "VYDAJ" THEN 1 ELSE 0 END) as 'Withdrawl'
FROM trans_a
GROUP BY account_id

""")

In [57]:
trans_a.head()

,account_id,operation,amount,number_of_transaction,Max_transaction,Min_transaction,AVG_transaction,TransactionMode,TransactionCharacterization,UsedForCredit,Withdrawl
0,1,VYBER,375174.5,239,12600.0,14.6,1569.767782,Withdrawl in cash,Payment for statement,102,137
1,2,None,3151479.3,478,42000.0,13.5,6593.052929,Other transaction,Intrest credited,152,305
2,3,VYBER,295021.8,117,11253.0,14.6,2521.553846,Withdrawl in cash,Payment for statement,46,71
3,4,VYBER,350971.4,186,5553.0,14.6,1886.943011,Withdrawl in cash,Payment for statement,65,121
4,5,None,166881.4,84,5100.0,3.5,1986.683333,Other transaction,Intrest credited,35,49


In [58]:
card_a = card

In [59]:
card_a = pysqldf("""
SELECT account_id, 
count(type) as number_of_cardtrypes
FROM card_a
GROUP BY account_id
""")

In [60]:
card_a.head()

,account_id,number_of_cardtrypes
0,9,1
1,19,1
2,41,1
3,42,1
4,51,1


In [61]:
Table3 = pd.merge(Table3,order_a, how = 'left', on = 'account_id')
Table3 = pd.merge(Table3,trans_a, how = 'left', on = 'account_id')
#Table3 = pd.merge(Table3,card_a, how = 'left', on = 'account_id')

In [62]:
Table3

,client_id,birth_number,district_id_x,birth_year,age,birth_month,birth_day,gender,district_name,region,...,operation,amount_y,number_of_transaction,Max_transaction,Min_transaction,AVG_transaction,TransactionMode,TransactionCharacterization,UsedForCredit,Withdrawl
0,1,706213,18,1970,29,12,13,F,Pisek,south Bohemia,...,VYBER,375174.5,239,12600.0,14.6,1569.767782,Withdrawl in cash,Payment for statement,102,137
1,2,450204,1,1945,54,2,4,M,Hl.m. Praha,Prague,...,None,3151479.3,478,42000.0,13.5,6593.052929,Other transaction,Intrest credited,152,305
2,3,406009,1,1940,59,10,9,F,Hl.m. Praha,Prague,...,None,3151479.3,478,42000.0,13.5,6593.052929,Other transaction,Intrest credited,152,305
3,4,561201,5,1956,43,12,1,M,Kolin,central Bohemia,...,VYBER,295021.8,117,11253.0,14.6,2521.553846,Withdrawl in cash,Payment for statement,46,71
4,5,605703,5,1960,39,7,3,F,Kolin,central Bohemia,...,VYBER,295021.8,117,11253.0,14.6,2521.553846,Withdrawl in cash,Payment for statement,46,71
5,6,190922,12,1919,80,9,22,M,Pribram,central Bohemia,...,VYBER,350971.4,186,5553.0,14.6,1886.943011,Withdrawl in cash,Payment for statement,65,121
6,7,290125,15,1929,70,1,25,M,Cesky Krumlov,south Bohemia,...,None,166881.4,84,5100.0,3.5,1986.683333,Other transaction,Intrest credited,35,49
7,8,385221,51,1938,61,2,21,F,Trutnov,east Bohemia,...,VYBER,647567.1,246,11600.0,14.6,2632.386585,Withdrawl in cash,Payment for statement,99,147
8,9,351016,60,1935,64,10,16,M,Prostejov,south Moravia,...,VYBER,1192039.9,130,33975.0,14.6,9169.537692,Withdrawl in cash,Payment for statement,51,79
9,10,430501,57,1943,56,5,1,M,Hodonin,south Moravia,...,VYBER,1712906.1,254,40800.0,14.6,6743.724803,Withdrawl in cash,Payment for statement,84,170


In [68]:
Table3.to_csv(r'C:\Users\abannem\Downloads\group_project.csv')

In [63]:
 WithdrawlsAVG = pysqldf("""
 SELECT district_name, Withdrawl
 FROM Table3
 WHERE withdrawl > 450
 """)


In [64]:
WithdrawlsAVG.nunique()

district_name    12
Withdrawl        10
dtype: int64

In [65]:
cumsum = np.cumsum(Table3["Withdrawl"])
Districts = Table3["district_name"]
Withdrawls = Table3["Withdrawl"]
unique_districts = (Table3["district_name"]).nunique()
print(unique_districts)

77


In [66]:
def top10 (x):
    unique_list = []
    
    for x in Table3[""]:
        if x > 150:

SyntaxError: unexpected EOF while parsing (<ipython-input-66-a4abfe060ece>, line 5)

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot

In [77]:
# Create a figure with a datetime type x-axis
fig = figure(title='Historical Bank Data',
             plot_height=400, plot_width=700,
             x_axis_label='District', y_axis_label='Withdarwl',
             x_minor_ticks=2, y_range=(0,1000),
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
fig.vbar(x=Districts, bottom=0, top=Withdrawls, 
         color='blue', width=0.75, 
         legend='Withdrawls')

# The cumulative sum will be a trend line
fig.line(x=Districts, y = cumsum, 
         color='gray', line_width=1,
         legend='Cumulative')

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

show(fig)

NameError: name 'figure' is not defined